In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from pandas import Series
import os 
from itertools import product

import warnings

#from modshogun import *

from sklearn import linear_model, decomposition
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, GroupKFold, LeaveOneGroupOut
from sklearn.externals.joblib import Parallel, delayed
from sklearn.preprocessing import RobustScaler, LabelEncoder, StandardScaler, Imputer, MinMaxScaler
from sklearn.pipeline import Pipeline

from CustomCVs import KFoldMixedSizes, StratifiedKFoldMixedSizes, StratifiedKFoldByGroups
#from evaluation_classifier import Evaluater

from time import time
from IPython.display import clear_output

#from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, IterativeSVD #, MICE

from six.moves import cPickle as pickle

import matplotlib.pyplot as plt

In [ ]:
# def create_rank_k_dataset(
#         n_rows=5,
#         n_cols=5,
#         k=3,
#         fraction_missing=0.1,
#         symmetric=False,
#         random_seed=0):
#     np.random.seed(random_seed)
#     x = np.random.randn(n_rows, k)
#     y = np.random.randn(k, n_cols)

#     XY = np.dot(x, y)

#     if symmetric:
#         assert n_rows == n_cols
#         XY = 0.5 * XY + 0.5 * XY.T

#     missing_raw_values = np.random.uniform(0, 1, (n_rows, n_cols))
#     missing_mask = missing_raw_values < fraction_missing

#     XY_incomplete = XY.copy()
#     # fill missing entries with NaN
#     XY_incomplete[missing_mask] = np.nan

#     return XY, XY_incomplete, missing_mask

# # create some default data to be shared across tests
# XY, XY_incomplete, missing_mask = create_rank_k_dataset(
#     n_rows=500,
#     n_cols=10,
#     k=3,
#     fraction_missing=0.25)

In [21]:
def create_correlated_dataset(cov_mat, mean_vec, n_obs = 2500):
    
    n_vars = cov_mat.shape[0]
    cov_mat = cov_mat + 0.5 * np.eye(n_vars) # regularize for stability

    try:
        L = np.linalg.cholesky(cov_mat)
        D = np.dot(L, np.random.uniform(0,1, (n_vars, n_obs)))
        D = D*mean_vec.T # get the scaling right
        return D
    
    except np.linalg.LinAlgError as err:
        print('Error ---- Cholesksy')
        return None
        
    

In [3]:
# test
cov_mat = np.array([[1, 0.7, 0.7, 0.5,],
             [0.7, 1, 0.95, 0.3],
             [0.7, 0.95, 1, 0.3],
             [0.5, 0.3, 0.3, 1]])
mean_vec=np.array([200, 200, 200, 200], ndmin=2)
D = create_correlated_dataset(cov_mat, mean_vec)
print(D.shape)
np.corrcoef(D)

(4, 2500)


array([[1.        , 0.70122939, 0.69552579, 0.49341817],
       [0.70122939, 1.        , 0.94805295, 0.28393164],
       [0.69552579, 0.94805295, 1.        , 0.27078981],
       [0.49341817, 0.28393164, 0.27078981, 1.        ]])

In [5]:
# Construct a cov matrix using a REAL dataset

data_dir="/data/rmthomas/HeteroSmallSample"
df = pd.read_csv(os.path.join(data_dir, "real_data.csv"))
df_numeric = df[df.columns[25:125]]

# cov_mat_overall = df_numeric.corr().values
# alpha = 0.2
# reg_cov_mat = cov_mat_overall + alpha*np.eye(100) # alpha makes the matrix well conditioned for Cholesky
# D_overall = create_correlated_dataset(reg_cov_mat)
# print(D_overall.shape)

In [6]:
n_features = 100 
g = df.groupby(['site', 'Dx', 'age_group'])[df.columns[25:25+n_features]]
Groups = list(g.indices.keys())

In [7]:
# Generate correlation matrices per set = (site, Dx, age_group)
corrs_per_set = g.corr().values.reshape(-1, n_features, n_features)
means_per_set = g.mean().values.reshape(-1, n_features)
#corrs_per_set[np.where(np.isnan(corrs_per_set))] = 0.5

In [8]:
feature_labels = [f'f{i}' for i in range(100)] # f1, f2 ...f100
data_cols = ['site', 'Dx', 'Age_group'] + feature_labels

In [22]:
sim_data_all = pd.DataFrame(columns=data_cols) # initialize a dataframe
group_template = pd.DataFrame(columns=['site', 'Dx', 'Age_group']) # initialize a dataframe

sim_data=[]

min_subj = 20
max_subj = 120
for corr_i in range(corrs_per_set.shape[0]):
    n_obs=np.random.choice(np.arange(min_subj, max_subj))
    D = create_correlated_dataset(corrs_per_set[corr_i], means_per_set[corr_i].reshape(1,n_features), n_obs=n_obs)

    if D is not None:
        sim_data_group = pd.DataFrame([list(Groups[corr_i])], columns=['site', 'Dx', 'Age_group'])
        sim_data_group = pd.concat([sim_data_group]*n_obs, ignore_index=True)
        
        sim_data_group_matrix = pd.DataFrame(D.T, columns=feature_labels)
        df_group = pd.concat([sim_data_group, sim_data_group_matrix], axis=1, ignore_index=False)
        sim_data_all = sim_data_all.append(df_group, ignore_index=True)
        sim_data.append(D)

Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy


In [23]:
sim_data_all.shape

(3539, 103)

In [24]:
[sim_data_all.mean().values, df_numeric.mean().values]

[array([ 9.55516852e+02,  2.84369960e+03,  3.80032262e+03,  1.52300732e+03,
         6.51628186e+03,  8.62601560e+03,  4.43821990e+03,  6.61318020e+02,
         1.97228524e+03,  5.04109614e+03,  5.56497297e+03,  2.24525309e+03,
         8.67420149e+03,  1.09170656e+04,  1.30077227e+05,  1.32604490e+03,
         7.72168105e+02,  3.30959867e+03,  1.84302243e+03,  4.95561525e+02,
         2.42811656e+02,  5.26837208e+03,  8.06528744e+03,  5.78127715e+03,
         3.74439074e+03,  1.60314644e+03,  8.34952123e+03,  4.73118773e+03,
         4.83321276e+03,  3.23568792e+03,  6.01269349e+03,  2.10754468e+03,
         1.16072919e+03,  2.73977626e+03,  1.14597437e+03,  2.24564370e+03,
         2.03493827e+03,  8.40104533e+03,  2.13537636e+03,  9.96202595e+03,
         8.17989486e+03,  1.61510954e+03,  1.26774243e+04,  1.61520624e+04,
         1.12877845e+04,  8.13278001e+03,  8.11409382e+03,  7.52341391e+02,
         8.40934720e+02,  2.33268614e+05,  1.78839079e+03,  1.27847011e+03,
         4.1

In [25]:
# Write it to file
sim_data_all.to_csv('simulated_data.csv')

In [ ]:
all_data = np.hstack(sim_data)
nvars, nsubjs = all_data.shape

In [ ]:
# Create data for each group
D_patients = create_correlated_dataset(cov_patients)
D_controls = create_correlated_dataset(cov_controls)

In [ ]:
from fancyimpute import MICE

In [ ]:
# https://www.kaggle.com/athi94/investigating-imputation-methods